# Final Project Part 3.1

## Group 7

Group Members: Anushri Bhagwath- anushri5, Srushti Karangale- shk10

In [1]:
#importing the required libraries
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [2]:
# Reading the data
NYC_cause_data = pd.read_csv('https://data.cityofnewyork.us/api/views/jb7j-dtam/rows.csv?accessType=DOWNLOAD',
                             na_values = {"Deaths": '.',
                                   "Death Rate": '.',
                                    "Age Adjusted Death Rate": '.'
                                         }).dropna()

Initially I had read the dataset without removing the '.' in the columns, however while doing some exploratory data analysis in Part 1, I faced some issues which made me realise that certain columns had dirty data which had to be cleaned.

In [3]:
# checking the few rows of dataframe created
NYC_cause_data.head()

,Year,Leading Cause,Sex,Race Ethnicity,Deaths,Death Rate,Age Adjusted Death Rate
0,2019,"Diseases of Heart (I00-I09, I11, I13, I20-I51)",Male,Hispanic,1603.0,136.802992,176.783287
1,2019,Malignant Neoplasms (Cancer: C00-C97),Male,Hispanic,1164.0,99.337918,121.581769
2,2019,Mental and Behavioral Disorders due to Acciden...,Male,Hispanic,423.0,36.099604,35.707896
3,2019,Diabetes Mellitus (E10-E14),Male,Hispanic,245.0,20.908754,25.409344
4,2019,Influenza (Flu) and Pneumonia (J09-J18),Male,Hispanic,182.0,15.532217,19.858210


Part 1 helped us know to that the column 'Sex' has 4 unique entries which should ideally just be 2 as its just either male or female. Hence we will need to clean this data and make sure that this column just has two unique values.

In [4]:
# replacing the F and M values to have only 2 unique sex values

NYC_cause_data['Sex'] = NYC_cause_data['Sex'].replace('F', 'Female')
NYC_cause_data['Sex'] = NYC_cause_data['Sex'].replace('M', 'Male')

In [5]:
#NYC_cause_data.Year.astype('int32')

In [6]:
#NYC_cause_data.Year = pd.to_datetime(NYC_cause_data["Year"], format="%Y")

In [7]:
#NYC_cause_data['Year']

In [8]:
# NYC_cause_data['Sex'].unique()

In [9]:
#NYC_cause_data.info()

In [10]:
#NYC_cause_data['Year'].unique()

#### Create a subset dataframe which consists of average deaths per Race Ethnicity, Sex and Year across New York City

In [11]:
# group the NYC_cause_data dataframe based on Race Ethnicity, Sex and Year and then find 
death_average = NYC_cause_data.groupby(['Race Ethnicity','Sex','Year'],as_index=False)['Deaths'].mean()

In [12]:
death_average

,Race Ethnicity,Sex,Year,Deaths
0,Asian and Pacific Islander,Female,2007,116.818182
1,Asian and Pacific Islander,Female,2008,121.454545
2,Asian and Pacific Islander,Female,2009,125.545455
3,Asian and Pacific Islander,Female,2010,125.545455
4,Asian and Pacific Islander,Female,2011,138.818182
...,...,...,...,...
69,White Non-Hispanic,Male,2010,1116.090909
70,White Non-Hispanic,Male,2011,1099.090909
71,White Non-Hispanic,Male,2012,1094.090909
72,White Non-Hispanic,Male,2013,1090.545455


In [13]:
# removing the column which has the ethnicity value 'Not Stated/Unknown'
death_average = death_average[death_average["Race Ethnicity"] != 'Not Stated/Unknown'] 

In [14]:
# filtering out the data from the dataframe according to the years. 
# Due to a huge dataset which resulted in a longer driven plot, we are only choosing and analyzing the data for the years between 2010 and 2015.
death_average = death_average[(death_average.Year > 2010) & (death_average.Year < 2015)  ]

In [15]:
# displaying the resulting dataframe
death_average

,Race Ethnicity,Sex,Year,Deaths
4,Asian and Pacific Islander,Female,2011,138.818182
5,Asian and Pacific Islander,Female,2012,144.545455
6,Asian and Pacific Islander,Female,2013,144.090909
7,Asian and Pacific Islander,Female,2014,155.090909
13,Asian and Pacific Islander,Male,2011,164.909091
14,Asian and Pacific Islander,Male,2012,168.727273
15,Asian and Pacific Islander,Male,2013,187.818182
16,Asian and Pacific Islander,Male,2014,197.636364
22,Black Non-Hispanic,Female,2011,669.181818
23,Black Non-Hispanic,Female,2012,671.272727


## Interactive Line chart in altair

Creating an interactive dashboard in which the first visual which is the "driver" plot is a line chart displaying the average deaths in New York city based on the Race Ethnicity. The second plot in the dashboard which is the "driven" plot is a grouped bar chart displaying the Average deaths in New York city based on the Years and Sex.

In [16]:
brush = alt.selection_interval(encodings=['x','y'])

#plot 1
Line_chart = alt.Chart(death_average).mark_line().encode(
    x='Race Ethnicity:N', # N is for nominal
    y='average(Deaths):Q',  # Q is for quantitative
    tooltip=(['Race Ethnicity:N','average(Deaths):Q'])
).add_selection(
    brush
)

#plot 2
bar_chart =  alt.Chart(death_average).mark_bar().encode(
  alt.Column('Year:Q'), 
  alt.X('Sex:N'),  # N is for nominal
  alt.Y('Deaths:Q', axis=alt.Axis(grid=False), title ='Average Deaths'), # Q is for quantitative
  alt.Color('Sex'),
  tooltip=(['Deaths:Q'])
  ).transform_filter(
    brush
)

#combining the two plots into a dashboard
dashboard = Line_chart.properties(width=300) |  bar_chart.properties(width=100)

In [17]:
# displaying the dashboard created
dashboard

alt.HConcatChart(...)

In [18]:
myJekyllDir = 'C:/Users/hp/anushri-bhagwath.github.io/assets/json/'

In [19]:
dashboard.save(myJekyllDir+"race_ethnicity_deathsNYC.json")

#### Prose:

- How to use the Dashboard:

The dashboard will give the user a holistic view regarding the average deaths in New York City during the years between 2010 and 2015 based on Race Ethnicity and Sex. The first plot which is also called the "driver" plot on the left side of the dashboard is a simple line chart which displays the average number of deaths in New York city on the y-axis and the Race-Ethnicity of the deceased people on the x-axis. Upon hovering on the four important points on the line visual for each Race-Ethnicity value, there is a tooltip shown which consists of the average death value for each Race-Ethnicity. The second plot on the right side of the dashboard which is also called the "driven" plot is a grouped bar chart displaying the average number of deaths in New York city on the y-axis and the Year data on the x-axis. The data on the x-axis is grouped based on the sex which is either Male or Female.The legend on the top right corder of the right visual helps us understand which color belongs to which sex. Hovering the cursor on the bars will show the average death values for every sex per year. Both the visuals on the dashboard are linked and the interaction between them can be achieved by selecting a portion of data on the left (driver) visual. The selection can be done by just doing a left-click on the left visual and dragging the cursor to the right. Depending on the area selected on the left(driver) visual, the visual on the right(driven plot) will change. Hence right side visual (driven plot) will display the average number of deaths per year based on sex for the selected Race-Ethnic group.

- list of 1 or more contextual datasets identified

We have identified two contexual datasets for our project. The names of the contexual datasets and a brief explanation is given below:

1. New York City Community Health Survey

A detailed information about the dataset can be found using the link https://catalog.data.gov/dataset/dohmh-community-health-survey-2010-2016. We plan to use this dataset in a csv format and its link is https://data.cityofnewyork.us/api/views/csut-3wpr/rows.csv?accessType=DOWNLOAD
The size of this dataset is only 3KB. This dataset contains the data regarding the health of New Yorkers, including neighborhood, borough, and citywide estimates on a broad range of chronic diseases and behavioral risk factors. This dataset might help us analyze our primary dataset which is 'New York City Leading Causes of Death' better by helping us analyze the various behavioral risk factors of New Yorkers and other factors.

2. NCHS - Leading Causes of Death: United States

A detailed information about the dataset can be found using the link https://catalog.data.gov/dataset/nchs-leading-causes-of-death-united-states. We plan to use this dataset in a csv format and its link is https://data.cdc.gov/api/views/bi63-dtpu/rows.csv?accessType=DOWNLOAD
The size of this dataset is 823KB. This dataset provides data regarding age-adjusted death rates for the 10 leading causes of death in the United States beginning in 1999. We feel that this dataset might help us better compare our primary dataset('New York City Leading Causes of Death') analysis with the analysis done for other states in the country. This might give us a holistic picture regarding the Leading Causes of Death in the United States.

3. The size of our primary dataset which is 'New York City Leading Causes of Death' is 107Kb. GitHub has a strict file limit of 100MB and our file is well within that limit. Also both our contexual datasets are 3kb and 823KB in size which is also within GitHub upload limit. Hence we won't have any issues uploading the file on GitHub. 